# Analytics & Descriptive statistics

Some question we will try to answer from the dataset related to our specific analysis
* Total number of answers in the questionnaire
* Geographical distributions
* If there is missing answers 
* SKILLS
    * Frequency of each skill
    * How are skills correlated with each others 
* JOPS 
    * Freauency of each job
    * How are jops related to each others 
* Ralation between the skills and the jobs
* What is the the specificty of each skill to a job 

In [1]:
DF_PATH = "../data/processed/1_preprocessed_df.pkl"
ROLES_PATH = "../data/raw/roles_short_names.csv" 
FIG_DIR = "../reports/figures"

NA_STRING = 'Not Specified'
TRANSPARENT_STRING = 'rgba(0, 0, 0, 0)'

# Columns we choose to work with that is related to our analysis 
ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageHaveWorkedWith',
                  'DatabaseHaveWorkedWith',
                  'PlatformHaveWorkedWith',
                  'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith',
                  'ToolsTechHaveWorkedWith',
                  'NEWCollabToolsHaveWorkedWith']

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import os

import plotly 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

from sklearn.manifold import TSNE
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import AgglomerativeClustering

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

# To display all columns and rows
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000



_________

## Read data and preprocess

In [3]:
# Read Data 
raw_df = pd.read_pickle(DF_PATH)
roles_names = pd.read_csv(ROLES_PATH, sep=';')

At first we will encode the data.

In [4]:
df = raw_df.copy()
encoded_dfs = {}

# Insiantiating the banarizer
banarizer = MultiLabelBinarizer()

for col in ROLE_COLS + TECH_COLS:
    encoded_df = pd.DataFrame(banarizer.fit_transform(df[col]),
                              columns = banarizer.classes_,
                              index = df[col].index)
    encoded_dfs[col] = encoded_df

In [25]:
# Merge the encoded dataframes
df = pd.concat(encoded_dfs, axis =1)
df.head(2)

DevType                           \
  Academic researcher Data or business analyst   
0                   0                        0   
1                   0                        0   

                                                                        \
  Data scientist or machine learning specialist Database administrator   
0                                             0                      0   
1                                             0                      0   

                                                                        \
  Designer DevOps specialist Developer, QA or test Developer, back-end   
0        0                 0                     0                   0   
1        0                 0                     0                   0   

                                                 \
  Developer, desktop or enterprise applications   
0                                             0   
1                                             0   

                                                                    \
  Developer, embedded applications or devices Developer, front-end   
0                                           0                    0   
1                                           0                    0   

                                                                       \
  Developer, full-stack Developer, game or graphics Developer, mobile   
0                     0                           0                 1   
1                     0                           0                 0   

                                                                          \
  Educator Engineer, data Engineer, site reliability Engineering manager   
0        0              0                          0                   0   
1        0              0                          0                   0   

                                                                           \
  Marketing or sales professional Other (please specify): Product manager   
0                               0                       0               0   
1                               0                       0               0   

                                                                               \
  Scientist Senior Executive (C-Suite, VP, etc.) Student System administrator   
0         0                                    0       0                    0   
1         0                                    0       0                    0   

  LanguageHaveWorkedWith                                                      \
                     APL Assembly Bash/Shell  C C# C++ COBOL Clojure Crystal   
0                      0        0          0  0  0   1     0       0       0   
1                      0        0          0  0  0   0     0       0       0   

                                                                           \
  Dart Delphi Elixir Erlang F# Go Groovy HTML/CSS Haskell Java JavaScript   
0    0      0      0      0  0  0      0        1       0    0          1   
1    0      0      0      0  0  0      0        0       0    0          1   

                                                                              \
  Julia Kotlin LISP Matlab Node.js Objective-C PHP Perl PowerShell Python  R   
0     0      0    0      0       0           1   1    0          0      0  0   
1     0      0    0      0       0           0   0    0          0      1  0   

                                           DatabaseHaveWorkedWith            \
  Ruby Rust SQL Scala Swift TypeScript VBA              Cassandra Couchbase   
0    0    0   0     0     1          0   0                      0         0   
1    0    0   0     0     0          0   0                      0         0   

                                                                        \
  DynamoDB Elasticsearch Firebase IBM DB2 MariaDB Microsoft SQL Server   
0        0             0        0       0       0                    0   
1        0 

## Display jop frequancy

In [18]:
jobs_freq = df['DevType'].copy().sum().sort_values().reset_index()

In [19]:
jobs_freq

,index,0
0,Marketing or sales professional,638
1,Scientist,2015
2,"Senior Executive (C-Suite, VP, etc.)",2103
3,"Developer, game or graphics",2112
4,Educator,2369
5,"Engineer, site reliability",2448
6,Academic researcher,2899
7,Product manager,3074
8,Other (please specify):,3545
9,"Developer, QA or test",3611


As we can see from the table above there is a big differance between the maximum and minimum value,
**So we have unbalanced classes**

## Display skills frequency

In [42]:
skills_freq = df.copy().drop("DevType", axis=1).sum().reset_index()
skills_freq.columns = ['group', 'skill', 'freq']

In [44]:
skills_freq.sort_values("freq",ascending=False)

,group,skill,freq
95,ToolsTechHaveWorkedWith,Git,68171
122,NEWCollabToolsHaveWorkedWith,Visual Studio Code,58026
19,LanguageHaveWorkedWith,JavaScript,53587
16,LanguageHaveWorkedWith,HTML/CSS,46259
29,LanguageHaveWorkedWith,Python,39792
33,LanguageHaveWorkedWith,SQL,38835
93,ToolsTechHaveWorkedWith,Docker,35644
47,DatabaseHaveWorkedWith,MySQL,35289
18,LanguageHaveWorkedWith,Java,29162
52,PlatformHaveWorkedWith,AWS,29138


In [52]:
fig = px.treemap(skills_freq, 
                 path=['group', 'skill'], 
                 color_continuous_scale='deep',
                 values='freq',color='freq')

# fig.update_layout(width=1400, height=700)
fig.show()

# Saving the figure in reports folder with .html for better visual
fig.write_html(os.path.join(FIG_DIR, 'treemap_skills_freq.html'))


In [65]:
fing = px.treemap(skills_freq,
                 path = ["group", "skill"], 
                 color_continuous_scale='deep',
                 values='freq',color='freq')
fing.show()

In [9]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ResponseId                    83439 non-null  int64  
 1   MainBranch                    83439 non-null  object 
 2   Employment                    83323 non-null  object 
 3   Country                       83439 non-null  object 
 4   US_State                      14920 non-null  object 
 5   UK_Country                    4418 non-null   object 
 6   EdLevel                       83126 non-null  object 
 7   Age1stCode                    83243 non-null  object 
 8   LearnCode                     83439 non-null  object 
 9   YearsCode                     81641 non-null  float32
 10  YearsCodePro                  61216 non-null  float32
 11  DevType                       83439 non-null  object 
 12  OrgSize                       60726 non-null  object 
 13  C

In [10]:
raw_df.sample().iloc[0]

ResponseId                                                                  81632
MainBranch                                 I am a student who is learning to code
Employment                                                     Student, full-time
Country                                                                     India
US_State                                                                      NaN
UK_Country                                                                    NaN
EdLevel                              Bachelor’s degree (B.A., B.S., B.Eng., etc.)
Age1stCode                                                          11 - 17 years
LearnCode                                                                [School]
YearsCode                                                                     6.0
YearsCodePro                                                                  NaN
DevType                                                                        []
OrgSize         

In [11]:
raw_df.MainBranch.unique()


array(['I am a developer by profession',
       'I am a student who is learning to code',
       'I am not primarily a developer, but I write code sometimes as part of my work',
       'I code primarily as a hobby',
       'I used to be a developer by profession, but no longer am',
       'None of these'], dtype=object)

In [12]:
raw_df[raw_df["MainBranch"] == "I am a developer by profession"]

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,"[Coding Bootcamp, Other online resources (ex: ...",NaN,NaN,"[Developer, mobile]",20 to 99 employees,EUR European Euro,4800.0,Monthly,"[C++, HTML/CSS, JavaScript, Objective-C, PHP, ...",[Swift],"[PostgreSQL, SQLite]",[SQLite],[],[],"[Laravel, Symfony]",[],[],[],[],[],"[PHPStorm, Xcode]","[Atom, Xcode]",MacOS,"[Call a coworker or friend, Visit Stack Overfl...",[Stack Overflow],Multiple times per day,Yes,A few times per month or weekly,"Yes, definitely",No,25-34 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[None of the above],[None of the above],Appropriate in length,Easy,62268.0
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,[],NaN,NaN,"[Developer, front-end]",100 to 499 employees,EUR European Euro,NaN,Monthly,"[JavaScript, TypeScript]","[JavaScript, TypeScript]",[],[],[],[],"[Angular, jQuery]","[Angular, jQuery]",[],[],[],[],[],[],Windows,"[Call a coworker or friend, Visit Stack Overfl...",[Stack Overflow],Daily or almost daily,Yes,Daily or almost daily,Neutral,No,35-44 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[I am deaf / hard of hearing],[],Appropriate in length,Neither easy nor difficult,NaN
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,[Friend or family member],17.0,10.0,"[Developer, desktop or enterprise applications...","Just me - I am a freelancer, sole proprietor, ...",GBP\tPound sterling,NaN,NaN,"[Bash/Shell, HTML/CSS, Python, SQL]","[Bash/Shell, HTML/CSS, Python, SQL]","[Elasticsearch, PostgreSQL, Redis]","[Cassandra, Elasticsearch, PostgreSQL, Redis]",[],[],[Flask],[Flask],"[Apache Spark, Hadoop, NumPy, Pandas]","[Hadoop, NumPy, Pandas]","[Docker, Git, Kubernetes, Yarn]","[Docker, Git, Kubernetes, Yarn]","[Atom, IPython/Jupyter, Notepad++, PyCharm, Vim]","[Atom, IPython/Jupyter, Notepad++, PyCharm, Vi...",Linux-based,"[Visit Stack Overflow, Go for a walk or other ...","[Stack Overflow, Stack Exchange]",Daily or almost daily,Yes,A few times per week,"Yes, somewhat",No,25-34 years old,[Man],No,[],[White or of European descent],[None of the above],[],Appropriate in length,Easy,NaN
8,9,I am a developer by profession,Employed part-time,India,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",18 - 24 years,[Coding Bootcamp],6.0,4.0,"[Developer, front-end]","10,000 or more employees",INR\tIndian rupee,NaN,Monthly,"[HTML/CSS, JavaScript]","[HTML/CSS, JavaScript]",[PostgreSQL],[PostgreSQL],[AWS],[],"[Django, FastAPI]","[Django, FastAPI]",[],[],"[Docker, Git]","[Docker, Git, Kubernetes]","[PyCharm, Sublime Text]","[PyCharm, Sublime Text]",Windows,"[Visit Stack Overflow, Google it, Panic]","[Stack Overflow, Stack Exchange]",A few times per week,Yes,Less than once per month or monthly,"Yes, definitely",No,25-34 years old,[Man],No,[],[South Asian],[],[I have a concentration and/or memory disorder...,Appropriate in length,Easy,NaN
9,10,I am a developer by profession,Employed fu